# Correspondence-driven plane-based M3C2 (PBM3C2)

<p style="color:red;"> <a style="font-weight: bold">
WARNING:</a> The implementation of this method is experimental and under active development.
</p>

In [ ]:
import py4dgeo

In this notebook, we present how the *Correspondence-driven plane-based M3C2* (PB-M3C2, [Zahs et al., 2022](#References)) algorithm for point cloud distance computation using the `py4dgeo` package.

The concept and method of PBM3C2 are explained in this scientific talk:
<a href="https://youtu.be/5pjkpajsRNU" target="_blank"><img src="https://github.com/3dgeo-heidelberg/py4dgeo/blob/main/doc/img/thumb_youtube_zahs_isprs2022.png?raw=true" alt="" width="400" /></a>


As PB-M3C2 is a learning algorithm, it requires user-labelled input data in the process. This input can either be provided through external tools or be generated using a simple graphical user interface. For the graphical user interface to work best from Jupyter notebooks, we select the `vtk` backend.

In [ ]:
py4dgeo.set_interactive_backend("vtk")

We will work on the same demonstrator data we used in the explanation of the [M3C2 algorithm](m3c2.ipynb):

In [ ]:
epoch0, epoch1 = py4dgeo.read_from_xyz(
    "plane_horizontal_t1.xyz", "plane_horizontal_t2.xyz"
)

Again, we instantiate an instance of the algorithm class. For now, we use only the defaults for its parameters and leave explanation of customization aspects for later.

In [ ]:
alg = py4dgeo.PBM3C2()

In a first step, PB-M3C2 will run a plane segmentation algorithm on the provided input point clouds. As a learning algorithm, it then requires user input about corresponding planes. `py4dgeo` offers two ways of doing this:
* You can export the segmentation data in XYZ format with four columns: `x`, `y` and `z` of the point cloud, as well as the `segment_id` of the segment the point is associated with. Using that data, you can determine correspondance using your favorite tools or existing workflows. Your input is again expected in a comma-separated text file (CSV). It should contain three columns: The `segment_id` from the first point cloud, the `segment_id` from the second point cloud and a value of `0` or `1` depending on whether the two segments matched. The APIs for this case are shown in this notebook.
* You can interactively build the correspondence information in an interactive session. For this, you can call `alg.build_labelled_similarity_features_interactively()`.

Here, we use the first method of using an external tool for labelling. This call will write a total of three files: The above mentioned XYZ files for both epochs, as well as a third file that contains the entire results of the segmentation process. This will allow you to start computation later on without rerunning the segmentation part of the algorithm. You can modify the default file names by passing them to the respective arguments.

In [ ]:
xyz_epoch0, xyz_epoch1, segment_id = alg.export_segmented_point_cloud_and_segments(
    epoch0=epoch0,
    epoch1=epoch1,
)

After doing the labelling using your preferred method, you can read it into `py4dgeo`. We pass the previously exported segmentation information and the externally produced CSV file to the traingin procedure. In this test case, we are distributing the labelled data with the test data:

In [ ]:
alg.training(
    extracted_segments_file_name="extracted_segments.seg",
    extended_y_file_name="testdata-labelling.csv",
)

We have now trained the algorithm using a `scikit-learn` classifier. By default, this is a random forest tree. We are now ready to compute the distances analoguous to how distances in standard M3C2 are calculated. This will run the prediction with the trained model and derive distance and uncertainty information from the results:

In [ ]:
distances, uncertainties = alg.compute_distances(epoch0=epoch0, epoch1=epoch1)

### References

* Zahs, V., Winiwarter, L., Anders, K., Williams, J.G., Rutzinger, M. & Höfle, B. (2022): Correspondence-driven plane-based M3C2 for lower uncertainty in 3D topographic change quantification. ISPRS Journal of Photogrammetry and Remote Sensing, 183, pp. 541-559. DOI: [10.1016/j.isprsjprs.2021.11.018](https://doi.org/10.1016/j.isprsjprs.2021.11.018).